# **Modelado: Volatilidad**

In [52]:
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.stats.diagnostic import acorr_ljungbox


In [53]:
btc = pd.read_csv(r'https://raw.githubusercontent.com/TawnyVTC/Proyectos_UN/refs/heads/main/2025/Deep_Learning/Data/btc_1d_with_volatility_and_lags.csv')
btc['Open time'] = pd.to_datetime(btc['Open time'], format='%Y-%m-%d')

# -----------------------
# 1. Crear lags de volatilidad
# -----------------------
for lag in [7, 14, 21, 28]:
    btc[f'Volatility_lag_{lag}'] = btc['Volatility'].shift(lag)

# -----------------------
# 2. Crear targets futuros (predicciones de volatilidad)
# -----------------------
for i in range(1, 8):  # t+1 a t+7
    btc[f'target_vol_t+{i}'] = btc['Volatility'].shift(-i)

# -----------------------
# 3. Definir features y targets
# -----------------------
features = [
    'Close', 'LogReturn', 'Volatility',
    'Volatility_lag_7', 'Volatility_lag_14', 'Volatility_lag_21', 'Volatility_lag_28'
]

targets = [f'target_vol_t+{i}' for i in range(1, 8)]

# -----------------------
# 4. Eliminar filas con NaN (por los shifts)
# -----------------------
btc = btc.dropna(subset=features + targets).reset_index(drop=True)

# -----------------------
# 5. Definir matrices de entrada y salida
# -----------------------
X = btc[features].values
y = btc[targets].values
dates = btc["Open time"].values

# -----------------------
# 6. Construcción del objeto timeSeries
# -----------------------
timeSeries = np.concatenate([X, y], axis=1)



In [54]:
from tsxv.splitTrainValTest import split_train_val_test_groupKFold
from sklearn.preprocessing import StandardScaler
import numpy as np

def _ensure_2d(arr):
    """
    Convierte el array a formato 2D.
    Si llega 1D → (n,1); si llega 3D → (n, timesteps*feats)
    """
    arr = np.asarray(arr)
    if arr.ndim == 1:
        return arr.reshape(-1, 1)
    if arr.ndim == 2:
        return arr
    if arr.ndim == 3:
        n, a, b = arr.shape
        return arr.reshape(n, a * b)
    raise ValueError(f"Array con ndim={arr.ndim} no soportado por esta función.")


def split_and_scale(timeSeries, n_steps_input=7, n_steps_forecast=7, n_steps_jump=1, target_col=2):
    X_list, y_list, Xcv_list, ycv_list, Xtest_list, ytest_list = split_train_val_test_groupKFold(
        timeSeries,
        n_steps_input,
        n_steps_forecast,
        n_steps_jump
    )

    X_train_scaled, X_val_scaled, X_test_scaled = [], [], []
    y_train_scaled, y_val_scaled, y_test_scaled = [], [], []
    scalers_x, scalers_y = [], []

    for fold in range(len(X_list)):
        X_train_raw = _ensure_2d(X_list[fold])
        X_val_raw   = _ensure_2d(Xcv_list[fold])
        X_test_raw  = _ensure_2d(Xtest_list[fold])

        # 👇 Aquí filtramos solo la variable objetivo (columna target)
        y_train_raw = _ensure_2d(y_list[fold])[:, target_col:target_col + n_steps_forecast]
        y_val_raw   = _ensure_2d(ycv_list[fold])[:, target_col:target_col + n_steps_forecast]
        y_test_raw  = _ensure_2d(ytest_list[fold])[:, target_col:target_col + n_steps_forecast]

        # Escaladores
        scaler_x = StandardScaler().fit(X_train_raw)
        scaler_y = StandardScaler().fit(y_train_raw)

        X_train_scaled.append(scaler_x.transform(X_train_raw))
        X_val_scaled.append(scaler_x.transform(X_val_raw))
        X_test_scaled.append(scaler_x.transform(X_test_raw))
        y_train_scaled.append(scaler_y.transform(y_train_raw))
        y_val_scaled.append(scaler_y.transform(y_val_raw))
        y_test_scaled.append(scaler_y.transform(y_test_raw))

        scalers_x.append(scaler_x)
        scalers_y.append(scaler_y)

        print(f"Fold {fold+1}: train {X_train_raw.shape} -> val {X_val_raw.shape} -> test {X_test_raw.shape}")

    return {
        'X_train': X_train_scaled,
        'X_val': X_val_scaled,
        'X_test': X_test_scaled,
        'y_train': y_train_scaled,
        'y_val': y_val_scaled,
        'y_test': y_test_scaled,
        'scalers_x': scalers_x,
        'scalers_y': scalers_y
    }


from tsxv.splitTrainValTest import split_train_val_test_groupKFold
from sklearn.preprocessing import StandardScaler

def _ensure_2d(arr):
    """
    Convierte el array a formato 2D.
    Si llega 1D → (n,1); si llega 3D → (n, timesteps*feats)
    """
    arr = np.asarray(arr)
    if arr.ndim == 1:
        return arr.reshape(-1, 1)
    if arr.ndim == 2:
        return arr
    if arr.ndim == 3:
        n, a, b = arr.shape
        return arr.reshape(n, a * b)
    raise ValueError(f"Array con ndim={arr.ndim} no soportado por esta función.")


def preparar_volatilidad(btc, n_lag, n_outputs=7):
    """
    Construye la matriz de características y targets usando volatilidad como variable objetivo.
    """
    # Variables base
    features = ['Volatility', 'Close', 'LogReturn',
                'Volatility_lag_7', 'Volatility_lag_14', 'Volatility_lag_21', 'Volatility_lag_28']
    targets = [f'target_t+{i}' for i in range(1, n_outputs+1)]

    X = btc[features].values
    y = btc[targets].values
    timeSeries = np.concatenate([X, y], axis=1)

    X_list, y_list, Xcv_list, ycv_list, Xtest_list, ytest_list = split_train_val_test_groupKFold(
        timeSeries, n_lag, n_outputs, 1
    )

    X_train_scaled, X_val_scaled, X_test_scaled = [], [], []
    y_train_scaled, y_val_scaled, y_test_scaled = [], [], []
    scalers_x, scalers_y = [], []

    for fold in range(len(X_list)):
        X_train_raw = _ensure_2d(X_list[fold])
        X_val_raw = _ensure_2d(Xcv_list[fold])
        X_test_raw = _ensure_2d(Xtest_list[fold])

        y_train_raw = _ensure_2d(y_list[fold])
        y_val_raw = _ensure_2d(ycv_list[fold])
        y_test_raw = _ensure_2d(ytest_list[fold])

        scaler_x = StandardScaler().fit(X_train_raw)
        scaler_y = StandardScaler().fit(y_train_raw)

        X_train_scaled.append(scaler_x.transform(X_train_raw))
        X_val_scaled.append(scaler_x.transform(X_val_raw))
        X_test_scaled.append(scaler_x.transform(X_test_raw))
        y_train_scaled.append(scaler_y.transform(y_train_raw))
        y_val_scaled.append(scaler_y.transform(y_val_raw))
        y_test_scaled.append(scaler_y.transform(y_test_raw))

        scalers_x.append(scaler_x)
        scalers_y.append(scaler_y)

    return {
        'X_train': X_train_scaled, 'X_val': X_val_scaled, 'X_test': X_test_scaled,
        'y_train': y_train_scaled, 'y_val': y_val_scaled, 'y_test': y_test_scaled,
        'scalers_x': scalers_x, 'scalers_y': scalers_y
    }


In [55]:
def test_independencia_residuos(residuos, lags=10):
    resultado = acorr_ljungbox(residuos, lags=[lags], return_df=True)
    return resultado['lb_pvalue'].iloc[0]

def _calc_metrics_per_horizon(y_true, y_pred):
    """
    y_true, y_pred: arrays (n_samples, n_horizons)
    Devuelve DataFrame con métricas por horizonte.
    """
    n_outputs = y_true.shape[1]
    rows = []
    for h in range(n_outputs):
        yt = y_true[:, h]
        yp = y_pred[:, h]
        mae = mean_absolute_error(yt, yp)
        mse = mean_squared_error(yt, yp)
        rmse = np.sqrt(mse)
        # MAPE con epsilon para evitar división por cero
        mape = np.mean(np.abs((yt - yp) / (np.abs(yt) + 1e-8))) * 100
        rows.append({'Horizonte': h+1, 'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'MAPE': mape})
    df = pd.DataFrame(rows)
    return df

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import os


def entrenar_y_guardar_modelo(data_dict, lag, n_outputs=7, epochs=100, batch_size=32, save_dir='models/volatilidad'):
    resultados = []
    for fold in range(len(data_dict['X_train'])):
        X_train, X_val, X_test = data_dict['X_train'][fold], data_dict['X_val'][fold], data_dict['X_test'][fold]
        y_train, y_val, y_test = data_dict['y_train'][fold], data_dict['y_val'][fold], data_dict['y_test'][fold]
        scaler_y = data_dict['scalers_y'][fold]

        model = Sequential([
            Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dense(n_outputs)
        ])
        model.compile(optimizer='adam', loss='mse')

        es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        model.fit(X_train, y_train, validation_data=(X_val, y_val),
                  epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[es])

        yhat_test = scaler_y.inverse_transform(model.predict(X_test))
        ytest_real = scaler_y.inverse_transform(y_test)

        df_metrics = _calc_metrics_per_horizon(ytest_real, yhat_test)
        rmse_prom = df_metrics['RMSE'].mean()
        resultados.append((fold, rmse_prom, model))

    # Guardar el mejor modelo
    best_fold, best_rmse, best_model = min(resultados, key=lambda x: x[1])
    model_dir = os.path.join(save_dir, f'lag_{lag}')
    os.makedirs(model_dir, exist_ok=True)
    model_path = os.path.join(model_dir, f'mejor_fold_lag_{lag}.keras')
    best_model.save(model_path)
    print(f"✅ Modelo lag {lag} guardado (fold {best_fold+1}) con RMSE {best_rmse:.4f}")

    return best_model


In [58]:

# Lags que quieres entrenar
lags = [7, 14, 21, 28]

for lag in lags:
    print(f"\n=== Entrenando modelo para lag {lag} ===")
    data_dict = split_and_scale(timeSeries)
    entrenar_y_guardar_modelo(data_dict, lag=lag)



=== Entrenando modelo para lag 7 ===
Fold 1: train (325, 98) -> val (108, 98) -> test (108, 98)
Fold 2: train (324, 98) -> val (108, 98) -> test (108, 98)
Fold 3: train (323, 98) -> val (108, 98) -> test (108, 98)
Fold 4: train (322, 98) -> val (108, 98) -> test (108, 98)
Fold 5: train (324, 98) -> val (108, 98) -> test (108, 98)
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
✅ Modelo lag 7 guardado (fold 4) con RMSE 0.0566

=== Entrenando modelo para lag 14 ===
Fold 1: train (325, 98) -> val (108, 98) -> test (108, 98)
Fold 2: train (324, 98) -> val (108, 98) -> test (108, 98)
Fold 3: train (323, 98) -> val (108, 98) -> test (108, 98)
Fold 4: train (322, 98) -> val (108, 98) -> test (108, 98)
Fold 5: train (324, 98) -> val (108, 98) -> test (108, 98)
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
4/4 ━━━━━━━━━━━━━━